In [1]:
import pandas as pd
import numpy as np
import os

# Prepare Demand

In [2]:
# Demand Size
demand_size_list = [i for i in range(100, 601, 50)]
# Demand Split Ratio (Intra Modal, %)
demand_split_ratio_list = [i for i in range(10, 91, 20)]

# Random Seed
random_seed_list = [3, 6, 9]

# Network Name
network_name = "11-500"

# Demand File Directory
demand_dir = f"data/demand/{network_name}/total/"

# Output Directory
output_dir = f"data/demand/{network_name}/amod/"

In [ ]:
for rs in random_seed_list:
    for ds in demand_size_list:
        for dsr in demand_split_ratio_list:
            demand_name = f"ds{ds}_dsr{dsr}_rs{rs}.csv"
            demand_path = os.path.join(demand_dir, demand_name)
            if not os.path.exists(demand_path):
                print(f"{demand_name} does not exist. Skipping...")
                continue

            demand_df = pd.read_csv(demand_path)

            # Add FleetPy Required Columns and Default Values
            demand_df['is_multimodal'] = False
            demand_df['modal_state'] = 0  # 0: AMoD, 3: FLM
            demand_df['transfer_station_ids'] = None
            demand_df['max_transfers'] = 0

            # Get All Left Inter-Modal Trips
            left_inter_modal_trips = demand_df[(demand_df['rq_type'] == 'inter') & (demand_df['subnetwork'] == 'left')].copy()
            # Update Columns for Left Inter-Modal Trips
            left_inter_modal_trips['is_multimodal'] = True
            left_inter_modal_trips['modal_state'] = 3  # FLM
            left_inter_modal_trips['transfer_station_ids'] = 'A;B' 
            left_inter_modal_trips['max_transfers'] = 1

            # Get All Right Inter-Modal Trips
            right_inter_modal_trips = demand_df[(demand_df['rq_type'] == 'inter') & (demand_df['subnetwork'] == 'right')].copy()
            # Update Columns for Right Inter-Modal Trips
            right_inter_modal_trips['is_multimodal'] = True
            right_inter_modal_trips['modal_state'] = 3  # FLM
            right_inter_modal_trips['transfer_station_ids'] = 'B;A'
            right_inter_modal_trips['max_transfers'] = 1

            # Update Demand DataFrame
            demand_df.update(left_inter_modal_trips)
            demand_df.update(right_inter_modal_trips)

            # Save Updated Demand DataFrame
            amod_demand_name = f"amod_ds{ds}_dsr{dsr}_rs{rs}.csv"
            output_path = os.path.join(output_dir, amod_demand_name)
            os.makedirs(output_dir, exist_ok=True)
            demand_df.to_csv(output_path, index=False)

# Prepare FleetPy Configuration Files

In [4]:
# Train Headway (min)
train_headway_list = [i for i in range(10, 31, 5)]

# MoD Fleet Size
mod_fleet_size_list = [i for i in range(30, 151, 20)]

# MaaS Platform Communication Strategy
maas_communication_strategy_list = ['default', 'TPCS']

# Random Seed
random_seed_list = [3, 6, 9]

# MoD Waiting Time Threshold (s)
mod_waiting_time_threshold_list = [300, 600, 900]
# MoD Detour Time Threshold (%)
mod_detour_time_threshold_list = [i for i in range(30, 101, 20)]

# Demand Size
demand_size_list = [i for i in range(100, 600, 50)]
# Demand Split Ratio (Intra Modal, %)
demand_split_ratio_list = [i for i in range(10, 91, 20)]

# Total Simulation Time (s)
total_sim_time = [0, 10800]  # 3 hours
# Warm-up Time (s)
warmup_time = 3600  # 1 hour
# Simulation Time Period (s)
time_period = [warmup_time, total_sim_time[1]+warmup_time]  # 1h + 3h

op_rh_reservation_max_routes = 6

amod_scenario_config_save_dirpath = "data/scenarios/fleetpy-configuration"
os.makedirs(amod_scenario_config_save_dirpath, exist_ok=True)

In [5]:
scenario_configs = []

for random_seed in random_seed_list:
    for fleet_size in mod_fleet_size_list:
        for demand_size in demand_size_list:
            for demand_split_ratio in demand_split_ratio_list:
                for broker_type in maas_communication_strategy_list:
                    for op_max_detour_time_factor in mod_detour_time_threshold_list:
                        for op_max_wait_time in mod_waiting_time_threshold_list:
                            for headway in train_headway_list:
                                scenario_name = f"amod-{demand_size}-{demand_split_ratio}-{fleet_size}-{broker_type}-{op_max_detour_time_factor}-{op_max_wait_time}-{headway}-{random_seed}-{time_period[0]}-{time_period[1]}"
                                rq_file = f"amod_ds{demand_size}_dsr{demand_split_ratio}_rs{random_seed}.csv"
                                op_fleet_composition = f'default_vehtype:{fleet_size}'
                                initial_state_scenario = f'vehicle_positions_{fleet_size}-{random_seed}'
                                gtfs_name = f"intercity-express-headway-{headway}"
                                sim_start_date = "20250101"
                                sim_broker_type = 'PTBrokerTPCS' if broker_type == 'TPCS' else 'PTBrokerDefault'

                                config_dict = {
                                    'scenario_name': scenario_name,
                                    'rq_file': rq_file,
                                    'op_fleet_composition': op_fleet_composition,
                                    'initial_state_scenario': initial_state_scenario,
                                    'gtfs_name': gtfs_name,
                                    'sim_start_date': sim_start_date,
                                    'op_rh_reservation_max_routes': op_rh_reservation_max_routes,
                                    'broker_type': sim_broker_type,
                                    'start_time': time_period[0],
                                    'end_time': time_period[1],
                                    'op_max_detour_time_factor': op_max_detour_time_factor,
                                    'op_max_wait_time': op_max_wait_time,
                                }

                                scenario_configs.append(config_dict)

df = pd.DataFrame(scenario_configs)
df.to_csv(os.path.join(amod_scenario_config_save_dirpath, f'amod_scenario_configs_20251006.csv'), index=False)

print(f"Total number of scenarios: {len(scenario_configs)}")

Total number of scenarios: 126000
